In [1]:
print "ready"

ready


In [30]:
import pandas as pd
import numpy as np
import os
import re
import csv
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import tfidf
from nltk.tokenize import word_tokenize
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [3]:
path='C:/Users/mom1/Desktop/571/'

In [4]:
mov=pd.read_csv(path+'mov3.csv')
del mov["Unnamed: 0"]
print mov.shape
print mov.columns
mov.head()

C:\Users\mom1\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1002784, 17)
Index([u'ID', u'title', u'IMDB URL', u'genre', u'director', u'cast',
       u'IMDB rating', u'mpaa', u'writer', u'certificates', u'runtimes',
       u'year', u'plot summary', u'plot keywords', u'type',
       u'full screen image URL', u'URL_ID'],
      dtype='object')


,ID,title,IMDB URL,genre,director,cast,IMDB rating,mpaa,writer,certificates,runtimes,year,plot summary,plot keywords,type,full screen image URL,URL_ID
0,361498.0,Death of a Dynasty,http://akas.imdb.com/title/tt0361498/,"Musical,Comedy",",Damon Dash",",(Ebon Moss-Bachrach)-(Dave Katz),(Capone Lee)...",3.9,"Rated R for pervasive language, some drug and ...",",Adam Moreno",[u'USA:R'],[u'92'],2003.0,"[u'A young journalist, David Katz, who writes ...","[u'hip-hop', u'scene-during-end-credits', u'af...",NaN,https://images-na.ssl-images-amazon.com/images...,361498.0
1,361499.0,The Secret Life of Racquel Darrian,http://akas.imdb.com/title/tt0361499/,Adult,",Toni English,Derrick Lane,Paul Thomas",",(Racquel Darrian)-(Herself),(Buck Adams)-(),(...",6.5,NaN,",Toni English",[u'USA:X'],[u'USA:147'],2001.0,"[u""Racquel Darrian's most complete 'best of' D...","[u'sex', u'lesbian-sex', u'hardcore', u'facial']",NaN,NaN,361499.0
2,361500.0,Delgo,http://akas.imdb.com/title/tt0361500/,"Animation,Adventure,Comedy,Fantasy,Romance",",Marc F. Adler,Jason Maurer",",(Freddie Prinze Jr.)-(Delgo),(Chris Kattan)-(...",4.4,Rated PG for sequences of fantasy action violence,",Patrick J. Cowan,Carl Dream,Jennifer Jones-Mi...","[u'Argentina:Atp', u'Australia:PG', u'Germany:...",[u'94'],2008.0,"[u'In an exotic divided land, Delgo, an advent...","[u'villainess', u'surrealism', u'one-word-titl...",NaN,https://images-na.ssl-images-amazon.com/images...,361500.0
3,361503.0,Devdas,http://akas.imdb.com/title/tt0361503/,Drama,",P.C. Barua",",(Nirmal Bannerjee)-(),(P.C. Barua)-(Devdas),(...",6.9,NaN,",P.C. Barua,Saratchandra Chatterjee",NaN,[u'India:141'],1935.0,NaN,[u'based-on-novel'],NaN,https://images-na.ssl-images-amazon.com/images...,361503.0
4,361505.0,Dharmatma,http://akas.imdb.com/title/tt0361505/,"Action,Romance,Thriller",",Feroz Khan",",(Prem Nath)-(Dharamdas 'Dharmatma'),(Feroz Kh...",6.8,NaN,",Kaushal Bharati,Kaushal Bharati,Kaushal Bharati",[u'India:A'],NaN,1975.0,"[u'Wealthy, powerful and influential Seth Dhar...","[u'beating', u'loss-of-loved-one', u'crime-bos...",NaN,https://images-na.ssl-images-amazon.com/images...,361505.0


In [5]:
columns=['ID', 'title','genre', 'director', 'cast',
       'writer','plot summary', 'plot keywords']
mov1=mov[columns]

Changing to lowercase, replacing "," and "u'" with space

In [7]:
pattern = re.compile(",")
cols=['title','genre','director','writer','cast','plot summary','plot keywords']
for i in cols:
    mov1[i]=mov1[i].str.lower()
    mov1[i]=mov1[i].replace(pattern," ")

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
pat=re.compile('(u\')')
mov1["plot summary"]=mov1["plot summary"].replace(pat," ")
mov1["plot keywords"]=mov1["plot keywords"].replace(pat," ")

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [9]:
mov1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords
0,361498.0,death of a dynasty,musical comedy,damon dash,(ebon moss-bachrach)-(dave katz) (capone lee)...,adam moreno,[ a young journalist david katz who writes f...,[ hip-hop' scene-during-end-credits' afric...
1,361499.0,the secret life of racquel darrian,adult,toni english derrick lane paul thomas,(racquel darrian)-(herself) (buck adams)-() (...,toni english,"[u""racquel darrian's most complete 'best of' d...",[ sex' lesbian-sex' hardcore' facial']
2,361500.0,delgo,animation adventure comedy fantasy romance,marc f. adler jason maurer,(freddie prinze jr.)-(delgo) (chris kattan)-(...,patrick j. cowan carl dream jennifer jones-mi...,[ in an exotic divided land delgo an adventu...,[ villainess' surrealism' one-word-title' ...
3,361503.0,devdas,drama,p.c. barua,(nirmal bannerjee)-() (p.c. barua)-(devdas) (...,p.c. barua saratchandra chatterjee,NaN,[ based-on-novel']
4,361505.0,dharmatma,action romance thriller,feroz khan,(prem nath)-(dharamdas 'dharmatma') (feroz kh...,kaushal bharati kaushal bharati kaushal bharati,[ wealthy powerful and influential seth dhara...,[ beating' loss-of-loved-one' crime-boss' ...


Add puntuations to stop words

In [12]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
stop_words.update(['!','u', '"' ,'#' ,'$' , '%' , '\'' , '(' , ')' , '*' , '+' , ',' , '-' ,'' ',' '',
                   '.' , '\\' , '/' , ':' , ';' ,'<' , '=' , '>' , '?' , '[' , ']' , '^' , '_', '`' , '~' ,'{', '}'  , '|' ])

Null Value Percentage

In [17]:
quantitive_columns = mov1.select_dtypes(include=[np.number]).columns
print len(quantitive_columns)
miss_quant_null=mov1[quantitive_columns].isnull().sum(axis=0)/mov1.shape[0]
miss_quant_null.sort_values(ascending=False)

1


ID    0.0
dtype: float64

In [16]:
qualitative_columns = mov1.select_dtypes(include=[object]).columns
print len(qualitative_columns)
miss_qual=mov1[qualitative_columns].isnull().sum(axis=0)/mov1.shape[0]
miss_qual.sort_values(ascending=False)

13


type                     0.999998
mpaa                     0.988379
certificates             0.834483
full screen image URL    0.782635
plot keywords            0.672186
plot summary             0.646314
runtimes                 0.547574
writer                   0.448091
director                 0.288216
cast                     0.103612
genre                    0.000000
IMDB URL                 0.000000
title                    0.000000
dtype: float64

Removing Null values from the columns of interest for tokenizing 

In [18]:
print mov1.shape
mov1=mov1.dropna(subset=["director"])
print mov1.shape
mov1=mov1.dropna(subset=["writer"])
print mov1.shape
mov1=mov1.dropna(subset=["plot summary"])
print mov1.shape
mov1=mov1.dropna(subset=["plot keywords"])
print mov.shape
mov1=mov1.dropna(subset=["cast"])
mov1.shape

(1002784, 13)
(713766, 13)
(488758, 13)
(255687, 13)
(1002784, 17)


(143036, 13)

In [12]:
mov1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_lem,title_tok
0,361498.0,death of a dynasty,musical comedy,damon dash,(ebon moss-bachrach)-(dave katz) (capone lee)...,adam moreno,[ a young journalist david katz who writes f...,[ hip-hop' scene-during-end-credits' afric...,"[death, dynasty]","[death, dynasti]","[death, of, a, dynasty]"
1,361499.0,the secret life of racquel darrian,adult,toni english derrick lane paul thomas,(racquel darrian)-(herself) (buck adams)-() (...,toni english,"[u""racquel darrian's most complete 'best of' d...",[ sex' lesbian-sex' hardcore' facial'],"[secret, life, racquel, darrian]","[secret, life, racquel, darrian]","[the, secret, life, of, racquel, darrian]"
2,361500.0,delgo,animation adventure comedy fantasy romance,marc f. adler jason maurer,(freddie prinze jr.)-(delgo) (chris kattan)-(...,patrick j. cowan carl dream jennifer jones-mi...,[ in an exotic divided land delgo an adventu...,[ villainess' surrealism' one-word-title' ...,[delgo],[delgo],[delgo]
3,361503.0,devdas,drama,p.c. barua,(nirmal bannerjee)-() (p.c. barua)-(devdas) (...,p.c. barua saratchandra chatterjee,NaN,[ based-on-novel'],[devdas],[devda],[devdas]
4,361505.0,dharmatma,action romance thriller,feroz khan,(prem nath)-(dharamdas 'dharmatma') (feroz kh...,kaushal bharati kaushal bharati kaushal bharati,[ wealthy powerful and influential seth dhara...,[ beating' loss-of-loved-one' crime-boss' ...,[dharmatma],[dharmatma],[dharmatma]


Tokenize, remove stop words and then stem "title"

In [11]:
mov1['title_nsw'] = np.empty((len(mov1), 0)).tolist()
mov1['title_lem'] = np.empty((len(mov1), 0)).tolist()
mov1["title_tok"] = [word_tokenize(i) for i in mov1["title"]]
for index,row in mov1.iterrows():
    [mov1.loc[index,"title_nsw"].append(w) for w in row["title_tok"] if not w in stop_words]
    for w in row["title_nsw"]:
        mov1.loc[index,"title_lem"].append(str(ps.stem(w)))

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [20]:
mov1=mov1.rename(columns={"title_lem":"title_stem"})

Tokenize, remove stop words and then stem "director"

In [28]:
mov1['director_nsw'] = np.empty((len(mov1), 0)).tolist()
mov1['director_stem'] = np.empty((len(mov1), 0)).tolist()
mov1["director_tok"] = [word_tokenize(i) for i in mov1["director"]] 
for index,row in mov1.iterrows():
    [mov1.loc[index,"director_nsw"].append(w) for w in row["director_tok"] if not w in stop_words]
    for w in row["director_nsw"]:
        w = unicode(w, errors='ignore')
        mov1.loc[index,"director_stem"].append(str(ps.stem(w)))

In [35]:
mov1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,title_tok,director_nsw,director_tok,director_stem,writer_nsw,writer_stem,writer_tok
0,361498.0,death of a dynasty,musical comedy,damon dash,(ebon moss-bachrach)-(dave katz) (capone lee)...,adam moreno,[ a young journalist david katz who writes f...,[ hip-hop' scene-during-end-credits' afric...,"[death, dynasty]","[death, dynasti]","[death, of, a, dynasty]","[damon, dash]","[damon, dash]","[damon, dash]","[adam, moreno]",[],"[adam, moreno]"
1,361499.0,the secret life of racquel darrian,adult,toni english derrick lane paul thomas,(racquel darrian)-(herself) (buck adams)-() (...,toni english,"[u""racquel darrian's most complete 'best of' d...",[ sex' lesbian-sex' hardcore' facial'],"[secret, life, racquel, darrian]","[secret, life, racquel, darrian]","[the, secret, life, of, racquel, darrian]","[toni, english, derrick, lane, paul, thomas]","[toni, english, derrick, lane, paul, thomas]","[toni, english, derrick, lane, paul, thoma]","[toni, english]",[],"[toni, english]"
2,361500.0,delgo,animation adventure comedy fantasy romance,marc f. adler jason maurer,(freddie prinze jr.)-(delgo) (chris kattan)-(...,patrick j. cowan carl dream jennifer jones-mi...,[ in an exotic divided land delgo an adventu...,[ villainess' surrealism' one-word-title' ...,[delgo],[delgo],[delgo],"[marc, f., adler, jason, maurer]","[marc, f., adler, jason, maurer]","[marc, f., adler, jason, maurer]","[patrick, j., cowan, carl, dream, jennifer, jo...",[],"[patrick, j., cowan, carl, dream, jennifer, jo..."
4,361505.0,dharmatma,action romance thriller,feroz khan,(prem nath)-(dharamdas 'dharmatma') (feroz kh...,kaushal bharati kaushal bharati kaushal bharati,[ wealthy powerful and influential seth dhara...,[ beating' loss-of-loved-one' crime-boss' ...,[dharmatma],[dharmatma],[dharmatma],"[feroz, khan]","[feroz, khan]","[feroz, khan]","[kaushal, bharati, kaushal, bharati, kaushal, ...",[],"[kaushal, bharati, kaushal, bharati, kaushal, ..."
6,361507.0,sanitarium,horror mystery sci-fi thriller,james eaves johannes roberts,(jeremy minns)-(dr. max warick) (kate copelan...,james eaves johannes roberts,[ a generation ago dr max warick was research...,[ hospital' medical' british' independen...,[sanitarium],[sanitarium],[sanitarium],"[james, eaves, johannes, roberts]","[james, eaves, johannes, roberts]","[jame, eav, johann, robert]","[james, eaves, johannes, roberts]",[],"[james, eaves, johannes, roberts]"


Tokenize and remove stop words from "writer"

In [33]:
mov1['writer_nsw'] = np.empty((len(mov1), 0)).tolist()
mov1["writer_tok"] = [word_tokenize(i) for i in mov1["writer"]]
for index,row in mov1.iterrows():
    [mov1.loc[index,"writer_nsw"].append(w) for w in row["writer_tok"] if not w in stop_words]

In [41]:
mov1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,title_tok,director_nsw,director_tok,director_stem,writer_nsw,writer_tok,cast_nsw
0,361498.0,death of a dynasty,musical comedy,damon dash,(ebon moss-bachrach)-(dave katz) (capone lee)...,adam moreno,[ a young journalist david katz who writes f...,[ hip-hop' scene-during-end-credits' afric...,"[death, dynasty]","[death, dynasti]","[death, of, a, dynasty]","[damon, dash]","[damon, dash]","[damon, dash]","[adam, moreno]","[adam, moreno]",[]
1,361499.0,the secret life of racquel darrian,adult,toni english derrick lane paul thomas,(racquel darrian)-(herself) (buck adams)-() (...,toni english,"[u""racquel darrian's most complete 'best of' d...",[ sex' lesbian-sex' hardcore' facial'],"[secret, life, racquel, darrian]","[secret, life, racquel, darrian]","[the, secret, life, of, racquel, darrian]","[toni, english, derrick, lane, paul, thomas]","[toni, english, derrick, lane, paul, thomas]","[toni, english, derrick, lane, paul, thoma]","[toni, english]","[toni, english]",[]
2,361500.0,delgo,animation adventure comedy fantasy romance,marc f. adler jason maurer,(freddie prinze jr.)-(delgo) (chris kattan)-(...,patrick j. cowan carl dream jennifer jones-mi...,[ in an exotic divided land delgo an adventu...,[ villainess' surrealism' one-word-title' ...,[delgo],[delgo],[delgo],"[marc, f., adler, jason, maurer]","[marc, f., adler, jason, maurer]","[marc, f., adler, jason, maurer]","[patrick, j., cowan, carl, dream, jennifer, jo...","[patrick, j., cowan, carl, dream, jennifer, jo...",[]
4,361505.0,dharmatma,action romance thriller,feroz khan,(prem nath)-(dharamdas 'dharmatma') (feroz kh...,kaushal bharati kaushal bharati kaushal bharati,[ wealthy powerful and influential seth dhara...,[ beating' loss-of-loved-one' crime-boss' ...,[dharmatma],[dharmatma],[dharmatma],"[feroz, khan]","[feroz, khan]","[feroz, khan]","[kaushal, bharati, kaushal, bharati, kaushal, ...","[kaushal, bharati, kaushal, bharati, kaushal, ...",[]
6,361507.0,sanitarium,horror mystery sci-fi thriller,james eaves johannes roberts,(jeremy minns)-(dr. max warick) (kate copelan...,james eaves johannes roberts,[ a generation ago dr max warick was research...,[ hospital' medical' british' independen...,[sanitarium],[sanitarium],[sanitarium],"[james, eaves, johannes, roberts]","[james, eaves, johannes, roberts]","[jame, eav, johann, robert]","[james, eaves, johannes, roberts]","[james, eaves, johannes, roberts]",[]


To handle unicode errors

In [6]:
cols=['genre','cast','plot summary','plot keywords']
for i in cols:
    mov1[i]=mov1[i].astype(unicode,raise_on_error=False)

Tokenize "cast"

In [11]:
mov1['cast_nsw'] = np.empty((len(mov1), 0)).tolist()
mov1.cast=mov1.cast.map(lambda x: x.decode('utf-8'))
mov1["cast_tok"] = [word_tokenize(i) for i in mov1["cast"]]
print "finished tokenizing"


finished tokenizing


NameError: name 'stop_words' is not defined

In [13]:
mov1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,title_tok,director_nsw,director_tok,director_stem,writer_nsw,writer_tok,cast_nsw,cast_tok
0,361498.0,death of a dynasty,musical comedy,damon dash,(ebon moss-bachrach)-(dave katz) (capone lee)...,adam moreno,[ a young journalist david katz who writes f...,[ hip-hop' scene-during-end-credits' afric...,"['death', 'dynasty']","['death', 'dynasti']","['death', 'of', 'a', 'dynasty']","['damon', 'dash']","['damon', 'dash']","['damon', 'dash']","['adam', 'moreno']","['adam', 'moreno']",[],"[(, ebon, moss-bachrach, ), -, (, dave, katz, ..."
1,361499.0,the secret life of racquel darrian,adult,toni english derrick lane paul thomas,(racquel darrian)-(herself) (buck adams)-() (...,toni english,"[u""racquel darrian's most complete 'best of' d...",[ sex' lesbian-sex' hardcore' facial'],"['secret', 'life', 'racquel', 'darrian']","['secret', 'life', 'racquel', 'darrian']","['the', 'secret', 'life', 'of', 'racquel', 'da...","['toni', 'english', 'derrick', 'lane', 'paul',...","['toni', 'english', 'derrick', 'lane', 'paul',...","['toni', 'english', 'derrick', 'lane', 'paul',...","['toni', 'english']","['toni', 'english']",[],"[(, racquel, darrian, ), -, (, herself, ), (, ..."
2,361500.0,delgo,animation adventure comedy fantasy romance,marc f. adler jason maurer,(freddie prinze jr.)-(delgo) (chris kattan)-(...,patrick j. cowan carl dream jennifer jones-mi...,[ in an exotic divided land delgo an adventu...,[ villainess' surrealism' one-word-title' ...,['delgo'],['delgo'],['delgo'],"['marc', 'f.', 'adler', 'jason', 'maurer']","['marc', 'f.', 'adler', 'jason', 'maurer']","['marc', 'f.', 'adler', 'jason', 'maurer']","['patrick', 'j.', 'cowan', 'carl', 'dream', 'j...","['patrick', 'j.', 'cowan', 'carl', 'dream', 'j...",[],"[(, freddie, prinze, jr., ), -, (, delgo, ), (..."
3,361505.0,dharmatma,action romance thriller,feroz khan,(prem nath)-(dharamdas 'dharmatma') (feroz kh...,kaushal bharati kaushal bharati kaushal bharati,[ wealthy powerful and influential seth dhara...,[ beating' loss-of-loved-one' crime-boss' ...,['dharmatma'],['dharmatma'],['dharmatma'],"['feroz', 'khan']","['feroz', 'khan']","['feroz', 'khan']","['kaushal', 'bharati', 'kaushal', 'bharati', '...","['kaushal', 'bharati', 'kaushal', 'bharati', '...",[],"[(, prem, nath, ), -, (, dharamdas, 'dharmatma..."
4,361507.0,sanitarium,horror mystery sci-fi thriller,james eaves johannes roberts,(jeremy minns)-(dr. max warick) (kate copelan...,james eaves johannes roberts,[ a generation ago dr max warick was research...,[ hospital' medical' british' independen...,['sanitarium'],['sanitarium'],['sanitarium'],"['james', 'eaves', 'johannes', 'roberts']","['james', 'eaves', 'johannes', 'roberts']","['jame', 'eav', 'johann', 'robert']","['james', 'eaves', 'johannes', 'roberts']","['james', 'eaves', 'johannes', 'roberts']",[],"[(, jeremy, minns, ), -, (, dr., max, warick, ..."


In [15]:
mov1.to_pickle('mov1.pickle')

In [16]:
mov1.to_csv('mov1.csv',encoding='utf-8')

In [ ]:
mov1['cast_nsw'] = np.empty((len(mov1), 0)).tolist()

Remove stop words from "cast_tok"

In [ ]:
for index,row in mov1.iterrows():
    [mov1.loc[index,"cast_nsw"].append(w) for w in row["cast_tok"] if not w in stop_words]
mov1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,...,writer_tok,cast_nsw,cast_tok,ps_nsw,ps_stem,ps_tok,pk_nsw,pk_stem,pk_tok,genre_tok
0,361498.0,death of a dynasty,musical comedy,damon dash,(ebon moss-bachrach)-(dave katz) (capone lee)...,adam moreno,[ a young journalist david katz who writes f...,[ hip-hop' scene-during-end-credits' afric...,"['death', 'dynasty']","[death, dynasti]",...,"['adam', 'moreno']","[ebon, moss-bachrach, dave, katz, capone, lee,...","[(, ebon, moss-bachrach, ), -, (, dave, katz, ...","[young, journalist, david, katz, writes, hip-h...","[young, journalist, david, katz, write, hip-ho...","[[, a, young, journalist, david, katz, who, wr...","[hip-hop, scene-during-end-credits, african-am...","[hip-hop, scene-during-end-credit, african-ame...","[[, hip-hop, ', scene-during-end-credits, ', a...","[musical, comedy]"
1,361499.0,the secret life of racquel darrian,adult,toni english derrick lane paul thomas,(racquel darrian)-(herself) (buck adams)-() (...,toni english,"[u""racquel darrian's most complete 'best of' d...",[ sex' lesbian-sex' hardcore' facial'],"['secret', 'life', 'racquel', 'darrian']","[secret, life, racquel, darrian]",...,"['toni', 'english']","[racquel, darrian, buck, adams, tom, byron, ch...","[(, racquel, darrian, ), -, (, herself, ), (, ...","['', racquel, darrian, 's, complete, 'best, dv...","['', racquel, darrian, 's, complet, 'best, dvd...","[[, u, '', racquel, darrian, 's, most, complet...","[sex, lesbian-sex, hardcore, facial]","[sex, lesbian-sex, hardcor, facial]","[[, sex, ', lesbian-sex, ', hardcore, ', facia...",[adult]
2,361500.0,delgo,animation adventure comedy fantasy romance,marc f. adler jason maurer,(freddie prinze jr.)-(delgo) (chris kattan)-(...,patrick j. cowan carl dream jennifer jones-mi...,[ in an exotic divided land delgo an adventu...,[ villainess' surrealism' one-word-title' ...,['delgo'],[delgo],...,"['patrick', 'j.', 'cowan', 'carl', 'dream', 'j...","[freddie, prinze, jr., delgo, chris, kattan, f...","[(, freddie, prinze, jr., ), -, (, delgo, ), (...","[exotic, divided, land, delgo, adventurous, dr...","[exot, divid, land, delgo, adventur, dreamer, ...","[[, in, an, exotic, divided, land, delgo, an, ...","[villainess, surrealism, one-word-title, compu...","[villai, surreal, one-word-titl, computer-anim...","[[, villainess, ', surrealism, ', one-word-tit...","[animation, adventure, comedy, fantasy, romance]"
3,361505.0,dharmatma,action romance thriller,feroz khan,(prem nath)-(dharamdas 'dharmatma') (feroz kh...,kaushal bharati kaushal bharati kaushal bharati,[ wealthy powerful and influential seth dhara...,[ beating' loss-of-loved-one' crime-boss' ...,['dharmatma'],[dharmatma],...,"['kaushal', 'bharati', 'kaushal', 'bharati', '...","[prem, nath, dharamdas, 'dharmatma, feroz, kha...","[(, prem, nath, ), -, (, dharamdas, 'dharmatma...","[wealthy, powerful, influential, seth, dharamd...","[wealthi, power, influenti, seth, dharamda, le...","[[, wealthy, powerful, and, influential, seth,...","[beating, loss-of-loved-one, crime-boss, fathe...","[beat, loss-of-loved-on, crime-boss, father-so...","[[, beating, ', loss-of-loved-one, ', crime-bo...","[action, romance, thriller]"
4,361507.0,sanitarium,horror mystery sci-fi thriller,james eaves johannes roberts,(jeremy minns)-(dr. max warick) (kate copelan...,james eaves johannes roberts,[ a generation ago dr max warick was research...,[ hospital' medical' british' independen...,['sanitarium'],[sanitarium],...,"['james', 'eaves', 'johannes', 'roberts']","[jeremy, minns, dr., max, warick, kate, copela...","[(, jeremy, minns, ), -, (, dr., max, warick, ...","[generation, ago, dr, max, warick, researching...","[gener, ago, dr, max, warick, research, myster...","[[, a, generation, ago, dr, max, warick, was, ...","[hospital, medical, british, independent-film]","[hospit, medic, british, independent-film]","[[, hospital, ', medical, ', british, ', indep...","[horror, mystery, sci-fi, thriller]"


Tokenize "plot summary"

In [18]:
mov1['ps_nsw'] = np.empty((len(mov1), 0)).tolist()
mov1['ps_stem'] = np.empty((len(mov1), 0)).tolist()
mov1["plot summary"]=mov1["plot summary"].map(lambda x: x.decode('utf-8'))
mov1["ps_tok"] = [word_tokenize(i) for i in mov1["plot summary"]]
print "finished tokenizing"

finished tokenizing


Removing stop words and then stemming "ps_tok"

In [20]:
for index,row in mov1.iterrows():
    [mov1.loc[index,"ps_nsw"].append(w) for w in row["ps_tok"] if not w in stop_words]
    for w in row["ps_nsw"]:
        mov1.loc[index,"ps_stem"].append(str(ps.stem(w)))
mov1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,...,director_nsw,director_tok,director_stem,writer_nsw,writer_tok,cast_nsw,cast_tok,ps_nsw,ps_stem,ps_tok
0,361498.0,death of a dynasty,musical comedy,damon dash,(ebon moss-bachrach)-(dave katz) (capone lee)...,adam moreno,[ a young journalist david katz who writes f...,[ hip-hop' scene-during-end-credits' afric...,"['death', 'dynasty']","['death', 'dynasti']",...,"['damon', 'dash']","['damon', 'dash']","['damon', 'dash']","['adam', 'moreno']","['adam', 'moreno']","[ebon, moss-bachrach, dave, katz, capone, lee,...","[(, ebon, moss-bachrach, ), -, (, dave, katz, ...","[young, journalist, david, katz, writes, hip-h...","[young, journalist, david, katz, write, hip-ho...","[[, a, young, journalist, david, katz, who, wr..."
1,361499.0,the secret life of racquel darrian,adult,toni english derrick lane paul thomas,(racquel darrian)-(herself) (buck adams)-() (...,toni english,"[u""racquel darrian's most complete 'best of' d...",[ sex' lesbian-sex' hardcore' facial'],"['secret', 'life', 'racquel', 'darrian']","['secret', 'life', 'racquel', 'darrian']",...,"['toni', 'english', 'derrick', 'lane', 'paul',...","['toni', 'english', 'derrick', 'lane', 'paul',...","['toni', 'english', 'derrick', 'lane', 'paul',...","['toni', 'english']","['toni', 'english']","[racquel, darrian, buck, adams, tom, byron, ch...","[(, racquel, darrian, ), -, (, herself, ), (, ...","['', racquel, darrian, 's, complete, 'best, dv...","['', racquel, darrian, 's, complet, 'best, dvd...","[[, u, '', racquel, darrian, 's, most, complet..."
2,361500.0,delgo,animation adventure comedy fantasy romance,marc f. adler jason maurer,(freddie prinze jr.)-(delgo) (chris kattan)-(...,patrick j. cowan carl dream jennifer jones-mi...,[ in an exotic divided land delgo an adventu...,[ villainess' surrealism' one-word-title' ...,['delgo'],['delgo'],...,"['marc', 'f.', 'adler', 'jason', 'maurer']","['marc', 'f.', 'adler', 'jason', 'maurer']","['marc', 'f.', 'adler', 'jason', 'maurer']","['patrick', 'j.', 'cowan', 'carl', 'dream', 'j...","['patrick', 'j.', 'cowan', 'carl', 'dream', 'j...","[freddie, prinze, jr., delgo, chris, kattan, f...","[(, freddie, prinze, jr., ), -, (, delgo, ), (...","[exotic, divided, land, delgo, adventurous, dr...","[exot, divid, land, delgo, adventur, dreamer, ...","[[, in, an, exotic, divided, land, delgo, an, ..."
3,361505.0,dharmatma,action romance thriller,feroz khan,(prem nath)-(dharamdas 'dharmatma') (feroz kh...,kaushal bharati kaushal bharati kaushal bharati,[ wealthy powerful and influential seth dhara...,[ beating' loss-of-loved-one' crime-boss' ...,['dharmatma'],['dharmatma'],...,"['feroz', 'khan']","['feroz', 'khan']","['feroz', 'khan']","['kaushal', 'bharati', 'kaushal', 'bharati', '...","['kaushal', 'bharati', 'kaushal', 'bharati', '...","[prem, nath, dharamdas, 'dharmatma, feroz, kha...","[(, prem, nath, ), -, (, dharamdas, 'dharmatma...","[wealthy, powerful, influential, seth, dharamd...","[wealthi, power, influenti, seth, dharamda, le...","[[, wealthy, powerful, and, influential, seth,..."
4,361507.0,sanitarium,horror mystery sci-fi thriller,james eaves johannes roberts,(jeremy minns)-(dr. max warick) (kate copelan...,james eaves johannes roberts,[ a generation ago dr max warick was research...,[ hospital' medical' british' independen...,['sanitarium'],['sanitarium'],...,"['james', 'eaves', 'johannes', 'roberts']","['james', 'eaves', 'johannes', 'roberts']","['jame', 'eav', 'johann', 'robert']","['james', 'eaves', 'johannes', 'roberts']","['james', 'eaves', 'johannes', 'roberts']","[jeremy, minns, dr., max, warick, kate, copela...","[(, jeremy, minns, ), -, (, dr., max, warick, ...","[generation, ago, dr, max, warick, researching...","[gener, ago, dr, max, warick, research, myster...","[[, a, generation, ago, dr, max, warick, was, ..."


Tokenize "plot keywords"

In [21]:
mov1['pk_nsw'] = np.empty((len(mov1), 0)).tolist()
mov1['pk_stem'] = np.empty((len(mov1), 0)).tolist()
mov1["plot keywords"]=mov1["plot keywords"].map(lambda x: x.decode('utf-8'))
mov1["pk_tok"] = [word_tokenize(i) for i in mov1["plot keywords"]]
print "finished tokenizing"

finished tokenizing


Removing stop words and then stemming "pk_tok"

In [23]:
for index,row in mov1.iterrows():
    [mov1.loc[index,"pk_nsw"].append(w) for w in row["pk_tok"] if not w in stop_words]
    for w in row["pk_nsw"]:
        mov1.loc[index,"pk_stem"].append(str(ps.stem(w)))
mov1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,...,writer_nsw,writer_tok,cast_nsw,cast_tok,ps_nsw,ps_stem,ps_tok,pk_nsw,pk_stem,pk_tok
0,361498.0,death of a dynasty,musical comedy,damon dash,(ebon moss-bachrach)-(dave katz) (capone lee)...,adam moreno,[ a young journalist david katz who writes f...,[ hip-hop' scene-during-end-credits' afric...,"['death', 'dynasty']","['death', 'dynasti']",...,"['adam', 'moreno']","['adam', 'moreno']","[ebon, moss-bachrach, dave, katz, capone, lee,...","[(, ebon, moss-bachrach, ), -, (, dave, katz, ...","[young, journalist, david, katz, writes, hip-h...","[young, journalist, david, katz, write, hip-ho...","[[, a, young, journalist, david, katz, who, wr...","[hip-hop, scene-during-end-credits, african-am...","[hip-hop, scene-during-end-credit, african-ame...","[[, hip-hop, ', scene-during-end-credits, ', a..."
1,361499.0,the secret life of racquel darrian,adult,toni english derrick lane paul thomas,(racquel darrian)-(herself) (buck adams)-() (...,toni english,"[u""racquel darrian's most complete 'best of' d...",[ sex' lesbian-sex' hardcore' facial'],"['secret', 'life', 'racquel', 'darrian']","['secret', 'life', 'racquel', 'darrian']",...,"['toni', 'english']","['toni', 'english']","[racquel, darrian, buck, adams, tom, byron, ch...","[(, racquel, darrian, ), -, (, herself, ), (, ...","['', racquel, darrian, 's, complete, 'best, dv...","['', racquel, darrian, 's, complet, 'best, dvd...","[[, u, '', racquel, darrian, 's, most, complet...","[sex, lesbian-sex, hardcore, facial]","[sex, lesbian-sex, hardcor, facial]","[[, sex, ', lesbian-sex, ', hardcore, ', facia..."
2,361500.0,delgo,animation adventure comedy fantasy romance,marc f. adler jason maurer,(freddie prinze jr.)-(delgo) (chris kattan)-(...,patrick j. cowan carl dream jennifer jones-mi...,[ in an exotic divided land delgo an adventu...,[ villainess' surrealism' one-word-title' ...,['delgo'],['delgo'],...,"['patrick', 'j.', 'cowan', 'carl', 'dream', 'j...","['patrick', 'j.', 'cowan', 'carl', 'dream', 'j...","[freddie, prinze, jr., delgo, chris, kattan, f...","[(, freddie, prinze, jr., ), -, (, delgo, ), (...","[exotic, divided, land, delgo, adventurous, dr...","[exot, divid, land, delgo, adventur, dreamer, ...","[[, in, an, exotic, divided, land, delgo, an, ...","[villainess, surrealism, one-word-title, compu...","[villai, surreal, one-word-titl, computer-anim...","[[, villainess, ', surrealism, ', one-word-tit..."
3,361505.0,dharmatma,action romance thriller,feroz khan,(prem nath)-(dharamdas 'dharmatma') (feroz kh...,kaushal bharati kaushal bharati kaushal bharati,[ wealthy powerful and influential seth dhara...,[ beating' loss-of-loved-one' crime-boss' ...,['dharmatma'],['dharmatma'],...,"['kaushal', 'bharati', 'kaushal', 'bharati', '...","['kaushal', 'bharati', 'kaushal', 'bharati', '...","[prem, nath, dharamdas, 'dharmatma, feroz, kha...","[(, prem, nath, ), -, (, dharamdas, 'dharmatma...","[wealthy, powerful, influential, seth, dharamd...","[wealthi, power, influenti, seth, dharamda, le...","[[, wealthy, powerful, and, influential, seth,...","[beating, loss-of-loved-one, crime-boss, fathe...","[beat, loss-of-loved-on, crime-boss, father-so...","[[, beating, ', loss-of-loved-one, ', crime-bo..."
4,361507.0,sanitarium,horror mystery sci-fi thriller,james eaves johannes roberts,(jeremy minns)-(dr. max warick) (kate copelan...,james eaves johannes roberts,[ a generation ago dr max warick was research...,[ hospital' medical' british' independen...,['sanitarium'],['sanitarium'],...,"['james', 'eaves', 'johannes', 'roberts']","['james', 'eaves', 'johannes', 'roberts']","[jeremy, minns, dr., max, warick, kate, copela...","[(, jeremy, minns, ), -, (, dr., max, warick, ...","[generation, ago, dr, max, warick, researching...","[gener, ago, dr, max, warick, research, myster...","[[, a, generation, ago, dr, max, warick, was, ...","[hospital, medical, british, independent-film]","[hospit, medic, british, independ

Tokenize "genre"

In [27]:
mov1["genre_tok"] = [word_tokenize(i) for i in mov1["genre"]]

Converting to list to create documents

In [11]:
cols=["title_stem","director_nsw","writer_nsw","cast_nsw","ps_stem","pk_stem","genre_tok"]
for i in cols:
    mov1[i]=mov1[i].str.translate(None,string.punctuation)
    mov1[i]=[item for item in mov1[i].str.split()]

Creating a copy of the dataset on local machine

In [13]:
mov1.to_csv('mov1.csv',encoding='utf-8')

In [29]:
mov1.shape

(143036, 25)

In [34]:
mov1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143036 entries, 0 to 143035
Data columns (total 25 columns):
ID               143036 non-null float64
title            143036 non-null object
genre            143036 non-null object
director         143036 non-null object
cast             143036 non-null object
writer           143036 non-null object
plot summary     143036 non-null object
plot keywords    143036 non-null object
title_nsw        143036 non-null object
title_stem       143036 non-null object
title_tok        143036 non-null object
director_nsw     143036 non-null object
director_tok     143036 non-null object
director_stem    143036 non-null object
writer_nsw       143036 non-null object
writer_tok       143036 non-null object
cast_nsw         0 non-null float64
cast_tok         143036 non-null object
ps_nsw           143036 non-null object
ps_stem          143036 non-null object
ps_tok           143036 non-null object
pk_nsw           143036 non-null object
pk_stem     

In [4]:
mov1=pd.read_csv(path+'mov1.csv')
del mov1["Unnamed: 0"]
mov1.shape

(143036, 25)

In [6]:
mov1.head()

,ID,title,genre,director,cast,writer,plot summary,plot keywords,title_nsw,title_stem,...,writer_tok,cast_nsw,cast_tok,ps_nsw,ps_stem,ps_tok,pk_nsw,pk_stem,pk_tok,genre_tok
0,361498.0,death of a dynasty,musical comedy,damon dash,(ebon moss-bachrach)-(dave katz) (capone lee)...,adam moreno,[ a young journalist david katz who writes f...,[ hip-hop' scene-during-end-credits' afric...,"['death', 'dynasty']","[death, dynasti]",...,"['adam', 'moreno']","[ebon, moss-bachrach, dave, katz, capone, lee,...","[(, ebon, moss-bachrach, ), -, (, dave, katz, ...","[young, journalist, david, katz, writes, hip-h...","[young, journalist, david, katz, write, hip-ho...","[[, a, young, journalist, david, katz, who, wr...","[hip-hop, scene-during-end-credits, african-am...","[hip-hop, scene-during-end-credit, african-ame...","[[, hip-hop, ', scene-during-end-credits, ', a...","[musical, comedy]"
1,361499.0,the secret life of racquel darrian,adult,toni english derrick lane paul thomas,(racquel darrian)-(herself) (buck adams)-() (...,toni english,"[u""racquel darrian's most complete 'best of' d...",[ sex' lesbian-sex' hardcore' facial'],"['secret', 'life', 'racquel', 'darrian']","[secret, life, racquel, darrian]",...,"['toni', 'english']","[racquel, darrian, buck, adams, tom, byron, ch...","[(, racquel, darrian, ), -, (, herself, ), (, ...","['', racquel, darrian, 's, complete, 'best, dv...","['', racquel, darrian, 's, complet, 'best, dvd...","[[, u, '', racquel, darrian, 's, most, complet...","[sex, lesbian-sex, hardcore, facial]","[sex, lesbian-sex, hardcor, facial]","[[, sex, ', lesbian-sex, ', hardcore, ', facia...",[adult]
2,361500.0,delgo,animation adventure comedy fantasy romance,marc f. adler jason maurer,(freddie prinze jr.)-(delgo) (chris kattan)-(...,patrick j. cowan carl dream jennifer jones-mi...,[ in an exotic divided land delgo an adventu...,[ villainess' surrealism' one-word-title' ...,['delgo'],[delgo],...,"['patrick', 'j.', 'cowan', 'carl', 'dream', 'j...","[freddie, prinze, jr., delgo, chris, kattan, f...","[(, freddie, prinze, jr., ), -, (, delgo, ), (...","[exotic, divided, land, delgo, adventurous, dr...","[exot, divid, land, delgo, adventur, dreamer, ...","[[, in, an, exotic, divided, land, delgo, an, ...","[villainess, surrealism, one-word-title, compu...","[villai, surreal, one-word-titl, computer-anim...","[[, villainess, ', surrealism, ', one-word-tit...","[animation, adventure, comedy, fantasy, romance]"
3,361505.0,dharmatma,action romance thriller,feroz khan,(prem nath)-(dharamdas 'dharmatma') (feroz kh...,kaushal bharati kaushal bharati kaushal bharati,[ wealthy powerful and influential seth dhara...,[ beating' loss-of-loved-one' crime-boss' ...,['dharmatma'],[dharmatma],...,"['kaushal', 'bharati', 'kaushal', 'bharati', '...","[prem, nath, dharamdas, 'dharmatma, feroz, kha...","[(, prem, nath, ), -, (, dharamdas, 'dharmatma...","[wealthy, powerful, influential, seth, dharamd...","[wealthi, power, influenti, seth, dharamda, le...","[[, wealthy, powerful, and, influential, seth,...","[beating, loss-of-loved-one, crime-boss, fathe...","[beat, loss-of-loved-on, crime-boss, father-so...","[[, beating, ', loss-of-loved-one, ', crime-bo...","[action, romance, thriller]"
4,361507.0,sanitarium,horror mystery sci-fi thriller,james eaves johannes roberts,(jeremy minns)-(dr. max warick) (kate copelan...,james eaves johannes roberts,[ a generation ago dr max warick was research...,[ hospital' medical' british' independen...,['sanitarium'],[sanitarium],...,"['james', 'eaves', 'johannes', 'roberts']","[jeremy, minns, dr., max, warick, kate, copela...","[(, jeremy, minns, ), -, (, dr., max, warick, ...","[generation, ago, dr, max, warick, researching...","[gener, ago, dr, max, warick, research, myster...","[[, a, generation, ago, dr, max, warick, was, ...","[hospital, medical, british, independent-film]","[hospit, medic, british, independent-film]","[[, hospital, ', medical, ', british, ', indep...","[horror, mystery, sci-fi, thriller]"


In [ ]:
reult= mov1.genre_tok.map(lambda x: set(x))
print len(result)

Creating documents

In [5]:
# %load tfidf.py
#!/usr/bin/env python

"""
The simplest TF-IDF library imaginable.

Add your documents as two-element lists `[docname, [list_of_words_in_the_document]]` with `addDocument(docname, list_of_words)`. Get a list of all the `[docname, similarity_score]` pairs relative to a document by calling `similarities([list_of_words])`.

See the README for a usage example.
"""

import sys
import os

class tfidf:
  def __init__(self):
    self.weighted = False
    self.documents = []
    self.corpus_dict = {}

  def addDocument(self, doc_name, list_of_words):
    # building a dictionary
    doc_dict = {}
    for w in list_of_words:
      doc_dict[w] = doc_dict.get(w, 0.) + 1.0
      self.corpus_dict[w] = self.corpus_dict.get(w, 0.0) + 1.0

    # normalizing the dictionary
    length = float(len(list_of_words))
    for k in doc_dict:
      doc_dict[k] = doc_dict[k] / length

    # add the normalized document to the corpus
    self.documents.append([doc_name, doc_dict])

  def similarities(self, list_of_words):
    """Returns a list of all the [docname, similarity_score] pairs relative to a list of words."""

    # building the query dictionary
    query_dict = {}
    for w in list_of_words:
      query_dict[w] = query_dict.get(w, 0.0) + 1.0

    # normalizing the query
    length = float(len(list_of_words))
    for k in query_dict:
      query_dict[k] = query_dict[k] / length

    # computing the list of similarities
    sims = []
    for doc in self.documents:
      score = 0.0
      doc_dict = doc[1]
      for k in query_dict:
        if k in doc_dict:
          score += (query_dict[k] / self.corpus_dict[k]) + (doc_dict[k] / self.corpus_dict[k])
      sims.append([doc[0], score])

    return sims

In [12]:
type(mov1.genre_tok[1])

list

In [6]:
table = tfidf.tfidf()
for index,row in mov1.iterrows():
    for i in row["genre_tok"]:
        table.addDocument(i,row["title_stem"]+row["title_stem"]+row["title_stem"]+row["director_nsw"]+row["director_nsw"]
                         +row["writer_nsw"]+row["writer_nsw"]+row["writer_nsw"]+row["cast_nsw"]
                         +row["cast_nsw"]+row["ps_stem"]+row["ps_stem"]+row["ps_stem"]+row["ps_stem"]+row["ps_stem"]
                         +row["pk_stem"]+row["pk_stem"]+row["pk_stem"]+row["pk_stem"]+row["pk_stem"]+row["pk_stem"]+row["pk_stem"])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\mom1\Anaconda2\lib\site-packages\IPython\core\ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\mom1\Anaconda2\lib\site-packages\IPython\core\ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "C:\Users\mom1\Anaconda2\lib\site-packages\IPython\core\ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "C:\Users\mom1\Anaconda2\lib\inspect.py", line 1049, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "C:\Users\mom1\Anaconda2\lib\inspect.py", line 1009, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "C:\Users\mom1\Anaconda2\lib\inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "C:\Users\mom1\Anaconda2\lib\inspect.py", line 497, i

IndexError: string index out of range

In [7]:
print len(table.documents)
#table.documents

363058


In [16]:
sample=mov1[0:5000].copy()

In [17]:
sample.to_csv('sample.csv')

In [18]:
print type(mov1.genre_tok[0])
mov1.genre_tok[0]

<type 'str'>


'[musical, comedy]'

In [19]:
mov1.genre_tok=mov1.genre_tok.map(lambda x: x.strip('[]'))

In [20]:
mov1["alll"]=mov1['title']+mov1['director']+mov1['cast']+mov1['writer']+mov1['plot keywords']+mov1['plot summary']

In [21]:
gens = {
    'action':    'action',
'adult': 'adult',
'adventure':'adventure',
'animation':'animation',
'biography':'biography',
'comedy':'comedy',
'crime':'crime',
'documentary':'documentary',
'drama':'drama',
'family':'family',
'fantasy':'fantasy',
'film-noir':'film-noir',
'game-show':'game-show',
'history':'history',
'horror':'horror',
'music':'music',
#'musical':'musical',
'mystery':'mystery',
'news':'news',
'news talk-show':'news talk-show',
'reality-tv':'reality-tv',
'romance':'romance',
'sci-fi':'sci-fi',
'short':'short',
'sport':'sport',
'talk-show':'talk-show',
'thriller':'thriller',
'war':'war',
'western':'western'
        }
for gen, pattern in gens.iteritems():
        mov1[gen]=mov1.genre_tok.str.contains(pattern)

In [23]:
print mov1.shape
mov1.columns

(143036, 54)


Index([u'ID', u'title', u'genre', u'director', u'cast', u'writer',
       u'plot summary', u'plot keywords', u'title_nsw', u'title_stem',
       u'title_tok', u'director_nsw', u'director_tok', u'director_stem',
       u'writer_nsw', u'writer_tok', u'cast_nsw', u'cast_tok', u'ps_nsw',
       u'ps_stem', u'ps_tok', u'pk_nsw', u'pk_stem', u'pk_tok', u'genre_tok',
       u'alll', u'family', u'adventure', u'fantasy', u'western', u'sport',
       u'biography', u'news talk-show', u'crime', u'romance', u'animation',
       u'music', u'comedy', u'war', u'sci-fi', u'talk-show', u'horror',
       u'adult', u'news', u'reality-tv', u'thriller', u'mystery', u'short',
       u'film-noir', u'drama', u'game-show', u'action', u'documentary',
       u'history'],
      dtype='object')

In [1]:
mov1.loc[0]

NameError: name 'mov1' is not defined

In [33]:
from sklearn.model_selection import train_test_split

target=[u'family', u'adventure', u'fantasy', u'western', u'sport',
       u'biography', u'news talk-show', u'crime', u'romance', u'animation',
       u'music', u'comedy', u'war', u'sci-fi', u'talk-show', u'horror',
       u'adult', u'news', u'reality-tv', u'thriller', u'mystery', u'short',
       u'film-noir', u'drama', u'game-show', u'action', u'documentary',
       u'history']
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                      ('clf', KNeighborsClassifier()),
 ])
X=mov1['alll']
y=mov1[target]
X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)
print X_train.shape
print y_train.shape

(114428L,)
(114428, 28)


In [35]:
text_clf = text_clf.fit(X_train,y_train)

In [40]:
def gen():
    text_clf.predict(X_test)

In [42]:
predicted = gen()

MemoryError: 

In [ ]:
np.mean(predicted == y_test)

In [ ]:
print(metrics.classification_report(y_test, predicted,target_names=target))

In [ ]:
X_train=mov1['alll']
y_train=mov1[target]
text_clf = text_clf.fit(X_train,y_train)

In [ ]:
mov2=pd.read_csv(path+'TestSet.txt')
del mov2["Unnamed: 0"]
print mov2.shape
print mov2.columns
mov2.head()

In [ ]:
mov2["alll"]=mov2['title']+mov2['director']+mov2['cast']+mov2['writer']+mov2['plot keywords']+mov2['plot summary']

In [ ]:
X_test=mov2['alll']
predicted = text_clf.predict(X_test)
print np.mean(predicted == y_test)
print(metrics.classification_report(y_test, predicted,target_names=target))